# Section B - Working with DataFrames and SQL

### B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.

In [1]:
from pprint import pprint
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .appName("Julie_Rajkumar_Amarwani_Lab3_Section-B")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

#### Loading CSV file

In [2]:
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.87:9000/parking-citations.csv')\
    .cache()

#### Verifying the data is loaded correctly.

In [3]:
data_frame.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

### B.2 Count the number of partitions in the underlying RDD.

In [4]:
data_frame.rdd.getNumPartitions()

11

### B.3 Print the schema for the DataFrame.

In [5]:
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



### B.4 Count the number of rows in the CSV file.

In [6]:
print("Total number of rows : ",data_frame.count())

Total number of rows :  9881842


### B.5 Drop the columns Agency Description, Agency, and Route.

In [7]:
data_frame_drop = data_frame.drop("Agency Description","Agency","Route")

print("Schema after deleting the above mentioned column \n")

data_frame_drop.printSchema()

Schema after deleting the above mentioned column 

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



### B.6 Find the mean fine amount (you need to convert the column to a float).

In [8]:
print("Before Converting to float type")
data_frame.select('Fine amount').show(10)

Before Converting to float type
+-----------+
|Fine amount|
+-----------+
|         50|
|         50|
|         58|
|       null|
|         93|
|         50|
|        163|
|        163|
|         93|
|         93|
+-----------+
only showing top 10 rows



- Converting to Float Type

In [9]:
from pyspark.sql.types import FloatType
import pyspark.sql.functions

DF_float = data_frame.withColumn("Fine amount", data_frame["Fine amount"].cast(FloatType()))

DF_float.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: float (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [10]:
print("After Converting the column to float type.\n")
DF_float.select('Fine amount').show(10)

After Converting the column to float type.

+-----------+
|Fine amount|
+-----------+
|       50.0|
|       50.0|
|       58.0|
|       null|
|       93.0|
|       50.0|
|      163.0|
|      163.0|
|       93.0|
|       93.0|
+-----------+
only showing top 10 rows



#### Mean of Fine Amount

In [11]:
from pyspark.sql.functions import col, avg

DF_float.agg(avg(col("Fine amount"))).show()
#DF_float.select('Fine amount').summary().show()


+----------------+
|avg(Fine amount)|
+----------------+
|70.1855354220642|
+----------------+



### B.7 Show the top 10 most frequent vehicle makes, and their frequencies.


In [14]:
DF_float.groupBy('Make')\
             .count()\
             .sort('count', ascending=False)\
             .show(10)

+----+-------+
|Make|  count|
+----+-------+
|TOYT|1633266|
|HOND|1113834|
|FORD| 860828|
|NISS| 709250|
|CHEV| 674422|
| BMW| 450909|
|MERZ| 402126|
|VOLK| 335618|
|HYUN| 304934|
|DODG| 290979|
+----+-------+
only showing top 10 rows



#### B.8 Let’s expand some abbreviations in the color column. Create a User Defined Function to create a new column, ‘color long’, mapping the original colors to their corresponding values in the dictionary below. If there is no key matching the original color, use the original color.

In [16]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
def color_long(shades):

    COLORS = {
    'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
    'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
    'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
    'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
    'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
    'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
    'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White',
    'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
    }

    if shades in COLORS:
        return COLORS[shades]
    else:
        return shades
    
# User-defined function. Input type is a string. 

udf_color_long = udf(color_long, StringType())
mapping_color = DF_float.withColumn('Color Long', udf_color_long('Color'))
mapping_color.select('Color','Color long').show(15)

+-----+----------+
|Color|Color long|
+-----+----------+
|   GY|      Gray|
|   WH|     White|
|   BK|     Black|
|   WH|     White|
|   BK|     Black|
|   GY|      Gray|
|   BL|      Blue|
|   BK|     Black|
|   BR|     Brown|
|   SI|    Silver|
|   WH|     White|
|   GO|      Gold|
|   BK|     Black|
|   BK|     Black|
|   BK|     Black|
+-----+----------+
only showing top 15 rows



### B.9 Using this new column, what’s the most frequent colour value for Hondas (HOND)?


In [17]:
HOND = mapping_color.filter(mapping_color['Make'] == 'HOND')
HOND.groupBy('Color Long')\
             .count()\
             .sort('count', ascending=False)\
             .show(1)

+----------+------+
|Color Long| count|
+----------+------+
|      Gray|266135|
+----------+------+
only showing top 1 row



In [18]:
spark_session.stop()